In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import os

from random import shuffle
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from glob import glob
from torchvision import transforms


In [3]:
torch.cuda.is_available()

True

In [4]:
def get_train_valid_data(base_dir, file_format=".jpg", valid_size=0.25):
    dirs_inside = {}
    for index, dir_name in enumerate(os.listdir(base_dir)):
        if os.path.isdir(os.path.join(base_dir, dir_name)):
            dirs_inside[dir_name] = index
            
    print(f'Found {len(dirs_inside.keys())} classes')
    
    train_data_x, train_data_y = [], []
    valid_data_x, valid_data_y = [], []
    
    for sub_dir in dirs_inside.keys():
        class_data = glob(os.path.join(base_dir, sub_dir) + f"\\*{file_format}", recursive=True) 
        class_samples_train = int(len(class_data) * (1 - valid_size))
        print(f'Found {len(class_data)} images for {sub_dir}')
        
        train_data_x += class_data[:class_samples_train]    
        valid_data_x += class_data[class_samples_train:]
    
    shuffle(train_data_x)
    shuffle(valid_data_x)
    train_data_y = [dirs_inside[file.split('\\')[1]] for file in train_data_x]
    valid_data_y = [dirs_inside[file.split('\\')[1]] for file in valid_data_x] 
    
    print(f"Train size is {len(train_data_x)}, validation size is {len(valid_data_x)}")
    return train_data_x, train_data_y, valid_data_x, valid_data_y

In [5]:
data_folder = "internship_data"
train_x, train_y, valid_x, valid_y = get_train_valid_data(base_dir=data_folder)

Found 2 classes
Found 50001 images for female
Found 50001 images for male
Train size is 75000, validation size is 25002


In [78]:
class MyImageDataset(Dataset):
    
    def __init__(self, files_list, base_dir, img_size=156, transform_compose=None):
        self.files = files_list
        self.classes = dict((cls, i) for i, cls in enumerate(os.listdir(base_dir)) if os.path.isdir(os.path.join(base_dir, cls)))
        self.size = img_size
        self.transforms = transforms.Compose([transforms.Resize((img_size, img_size)), 
                                              transforms.ToTensor(),]) if transform_compose is None else transform_compose  
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, key):
        pillow_img = Image.open(self.files[key])
        x = self.transforms(pillow_img)
        y = [0. for i in range(len(self.classes))]
        y[self.classes[self.files[key].split('\\')[1]]] = 1.
        return x, y

In [7]:
class CudaLoader:
    
    def __init__(self, DataLoader, shape):
        self.dl = DataLoader
        self.dev = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
        self.img_shape = shape  # Must be (C, H, W)
        print('Device is', self.dev)
    
    def __len__(self):
        return len(self.dl)
    
    def __iter__(self):
        iter_dl = iter(self.dl)
        for xb, yb in iter_dl:
            yield xb, yb  # self.preprocess(xb, yb)
    
    def preprocess(self, xb, yb):
        return xb.view(-1, self.img_shape[0], self.img_shape[1], self.img_shape[2]).to(self.dev), yb.to(self.dev)

In [8]:
IMG_SIZE = 156
train_ds = MyImageDataset(train_x, base_dir=data_folder, img_size=IMG_SIZE)
valid_ds = MyImageDataset(valid_x, base_dir=data_folder, img_size=IMG_SIZE)

In [9]:
tensor_dl_train = DataLoader(train_ds, batch_size=32, shuffle=True)
tensor_dl_valid = DataLoader(valid_ds)

In [10]:
cuda_train = CudaLoader(tensor_dl_train, shape=(3, IMG_SIZE, IMG_SIZE))
cuda_valid = CudaLoader(tensor_dl_valid, shape=(3, IMG_SIZE, IMG_SIZE))

Device is cuda:0
Device is cuda:0


## Create model

In [92]:
from torchvision.models import vgg16

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.features.parameters():
            param.requires_grad = False
            
def get_vgg_model(num_classes, feature_extracting=True):
    """Get saved vgg model with fixed parameters if feature_extracting = True
        feature_extracting: bool
    returns: model, input_size"""
    input_size = 224
    model = torch.load("vgg16_model.pt")
    set_parameter_requires_grad(model, feature_extracting=True)
    
    
    num_features = model.classifier[6].in_features
    features = list(model.classifier.children())[:-1] # Remove last layer
    features.extend([nn.Linear(num_features, num_classes)]) # Add our layer with activation
    model.classifier = nn.Sequential(*features) # Replace the model classifier

    return model, input_size

In [79]:
torch.cuda.empty_cache()

In [93]:
vgg_model, img_size = get_vgg_model(num_classes=2, feature_extracting=True)
vgg_model.cuda()

vgg_transforms = transforms.Compose([transforms.Resize((img_size, img_size)), 
                                     transforms.ToTensor(), 
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_ds_vgg = MyImageDataset(train_x, base_dir=data_folder, img_size=img_size, transform_compose=vgg_transforms)
valid_ds_vgg = MyImageDataset(valid_x, base_dir=data_folder, img_size=img_size, transform_compose=vgg_transforms)

vgg_dl_train = DataLoader(train_ds_vgg, batch_size=32, shuffle=True)
vgg_dl_valid = DataLoader(valid_ds_vgg)

In [96]:
optimizer_vgg = torch.optim.Adam(vgg_model.parameters(), lr=1e-3)
loss_function_vgg = nn.CrossEntropyLoss()

In [94]:
for i in range(3):
    print('file:', train_ds_vgg.files[i], 'class', train_ds_vgg[i][1])

file: internship_data\female\017263.jpg class [1.0, 0.0]
file: internship_data\male\142672.jpg class [0.0, 1.0]
file: internship_data\female\143205.jpg class [1.0, 0.0]


In [103]:
it = iter(vgg_dl_train)
xb, yb = next(it)

### My simple Neural net

In [53]:
class CNN_Net(nn.Module):
    
    def __init__(self, input_channels=3, output_size=1):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=428, kernel_size=3)
        
        self.fc1 = nn.Linear(428*3*3, 1024)  # sizes from c5
        self.fc2 = nn.Linear(1024, 128)
        self.out = nn.Linear(128, output_size)
        
        self.flatten_size = None
    
    def forward(self, x):
        c1 = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        c2 = F.max_pool2d(F.relu(self.conv2(c1)), (2, 2))
        c3 = F.max_pool2d(F.relu(self.conv3(c2)), (2, 2))
        c4 = F.max_pool2d(F.relu(self.conv4(c3)), (2, 2))
        c5 = F.max_pool2d(F.relu(self.conv5(c4)), (2, 2))
        
        if self.flatten_size is None:
            self.flatten_size = c5.shape[1] * c5.shape[2] * c5.shape[3]
            print(c5.shape, self.flatten_size)
            
        c5 = c5.view(-1, self.flatten_size)
        f1 = F.relu(self.fc1(c5))
        f2 = F.relu(self.fc2(f1))
        out = self.out(f2)
        return out

In [58]:
model = CNN_Net()
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)
loss_function = nn.BCEWithLogitsLoss()

In [59]:
print(next(model.parameters()).device)

cuda:0


In [114]:
def accuracy(pred_tensor, target):
    pred_classes = torch.argmax(pred_tensor, dim=1)
#     true_classes = torch.argmax(target, dim=1)
    equal_classes = (pred_classes == target).float()
    return torch.mean(equal_classes)

In [117]:
def train(model, epochs, optimizer, loss_func, train_loader, valid_loader=None):
    train_history = [[], []]
    valid_history = [[], []]
    for ep in range(epochs):
        ep_train_loss, ep_train_acc = [], []
        
        for i, (xb, yb) in enumerate(train_loader):
            optimizer.zero_grad()
            
            yb = yb[1].long().cuda()
            
            preds = model(xb.cuda())
            loss = loss_func(preds, yb)
            loss.backward()
            optimizer.step()
            
            acc = accuracy(preds, yb).item()
            ep_train_acc.append(acc)
            ep_train_loss.append(loss.item())
            
            print(acc, loss.item())
            if i == 200:
                break
            
        # mean accuracy and loss for current epoch
        train_history[0].append(np.array(ep_train_acc).mean())
        train_history[1].append(np.array(ep_train_loss).mean())
        
        if valid_loader is not None:
            with torch.no_grad():
                eval_acc, eval_loss = [], []
                for xb, yb in valid_loader:
                    preds = model(xb.cuda())
                    eval_loss.append(loss_func(preds, yb.float().cuda()).item())
                    eval_acc.append(accuracy(preds, yb.float().cuda()).item())
                # mean accuracy and loss for current epoch with validation data
                valid_history[0].append(np.array(eval_acc).mean())
                valid_history[1].append(np.array(eval_loss).mean())
        
        print(f'Epoch {ep}. acc:', round(np.array(ep_train_acc).mean(), 5), 'loss:', round(np.array(ep_train_loss).mean(), 5))
    
    return train_history, valid_history

In [118]:
train_hist, valid_hist = train(vgg_model, epochs=1, optimizer=optimizer_vgg, loss_func=loss_function_vgg, train_loader=vgg_dl_train, valid_loader=None) # 

0.875 0.2517974078655243
0.96875 0.13388067483901978
0.84375 0.4474097490310669
0.96875 0.09961051493883133
0.90625 0.6599254608154297
0.75 0.7901031970977783
0.9375 0.20271745324134827
0.96875 0.05812651664018631
0.96875 0.15721526741981506
1.0 0.04517175629734993
0.9375 0.4244026839733124
0.9375 0.23847399652004242
0.96875 0.05725773796439171
0.9375 0.1249626874923706
0.90625 0.20293526351451874
0.96875 0.10476718097925186
0.875 0.18727894127368927
0.84375 0.32219773530960083
0.9375 0.2590535283088684
0.9375 0.41855326294898987
0.90625 0.2780841290950775
0.90625 0.21175704896450043
0.90625 0.13331152498722076
0.875 0.5316754579544067
0.96875 0.08402818441390991
0.9375 0.0998460203409195
0.875 0.32924968004226685
0.875 0.3359799087047577
0.9375 0.36226704716682434
0.84375 0.47854018211364746
0.96875 0.07070047408342361
0.9375 0.1947792023420334
0.875 0.35964369773864746
0.96875 0.40433380007743835
0.90625 0.1947077512741089
0.8125 0.6496139764785767
0.875 0.4595957100391388
0.90625 0.

In [121]:
def validate_model(model, data_loader, num_samples=100, cuda_model=True):
    predictions, real_classes = [], []
    for sample_index, (xb, yb) in enumerate(data_loader):
        with torch.no_grad():
            if cuda_model:
                preds = vgg_model(xb.cuda())
            else:
                preds = vgg_model(xb)
                
        predictions.append(torch.argmax(preds.cpu()).item())
        real_classes.append(yb[1].item())
        if sample_index == num_samples:
            break
            
    predictions = np.array(predictions, dtype=int)
    real_classes = np.array(real_classes, dtype=int)
    print("accuracy is", np.mean((predictions==real_classes).astype(float)) )
    return predictions, real_classes

In [122]:
preds, real = validate_model(vgg_model, vgg_dl_valid, num_samples=1000)

accuracy is 0.9340659340659341


In [30]:
for i, (xb, yb) in enumerate(vgg_dl_valid):
    with torch.no_grad():
        preds = vgg_model(xb.cuda())
    preds = torch.round(preds)
    print(f'Predictid class {preds.squeeze().item()}, real class {yb.item()}')
    if i == 2:
        break

Predictid class 1.0, real class 1
Predictid class 1.0, real class 1
Predictid class 0.0, real class 0
